# Simple Marlowe Transactions in the Babbage Era

**Executive Summary**

This notebook demonstrates submitting Marlowe transactions in the Babbage Era using the low-level functions of `marlowe-cli` to create the Marlowe's Plutus script and the datums and redeemers needed when running Marlowe transactions. It uses `cardano-cli` to build, sign, and submit the transactions.


**Highlights**

-   Marlowe contracts in the Babbage Era.
-   Buildinng Marlowe transactions without using the higher-level functions of `marlowe-cli` or the Marlowe backend.

## 0. Preliminaries

Record the versions of the tools used.

In [1]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


## 1. Select the network

We use the `preview` network. See [its configuration files](https://book.world.dev.cardano.org/environments.html#preview-testnet) and use [its faucet](https://faucet.preview.world.dev.cardano.org/basic-faucet).

In [3]:
MAGIC=2

In [4]:
export CARDANO_NODE_SOCKET_PATH=node.socket

Find the current tip

In [5]:
TIP=$(cardano-cli query tip --testnet-magic "$MAGIC" | jq .slot)
echo "The tip of the blockchain is at slot $TIP."

The tip of the blockchain is at slot 1005544.


Retrieve the protocol parameters.

In [6]:
cardano-cli query protocol-parameters --testnet-magic "$MAGIC" --out-file simple-babbage.protocol

Previously, we empirically determined the slotting parameters for this network.

In [7]:
SLOT_OFFSET=$((1660003200 * 1000))  # POSIX milliseconds for slot zero
SLOT_SPACING=1000                   # milliseconds per slot

In [8]:
echo $SLOT_OFFSET

1660003200000


Convert a slot number to POSIX milliseconds.

In [9]:
function slot2time () {
  echo $(($1 * SLOT_SPACING + SLOT_OFFSET))
}

Convert POSIX milliseconds to a slot number.

In [10]:
function time2slot () {
  echo $((($1 - SLOT_OFFSET) / SLOT_SPACING))
}

Test the time conversion functions.

In [11]:
NOW=$(slot2time $TIP)
echo $NOW

1661008744000


In [12]:
echo "Compare $(time2slot $NOW) to $TIP."

Compare 1005544 to 1005544.


Define some time constants, for convenience.

In [13]:
SECOND=1000              # milliseconds per second
MINUTE=$((60 * SECOND))  # milliseconds per minute
HOUR=$((60 * MINUTE))    # milliseconds per hour
DAY=$((24 * HOUR))       # milliseconds per day

Define a constant for Ada.

In [14]:
ADA=1000000  # Lovelace per Ada

## 2. Select the signing and payment keys

Use pre-existing stakeless keys for this example.

In [15]:
PAYMENT_SKEY=payment.skey
PAYMENT_VKEY=payment.vkey

Compute the address.

In [16]:
ADDRESS_P=$(cardano-cli address build --testnet-magic $MAGIC --payment-verification-key-file $PAYMENT_VKEY)
echo $ADDRESS_P

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


Compute the public key hash.

In [17]:
PUBKEYHASH_P=$(cardano-cli address key-hash --payment-verification-key-file $PAYMENT_VKEY)
echo $PUBKEYHASH_P

0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07


Find a UTxO that we can use the fund the contract.

In [18]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_P

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
1f573374146f20b223473329c267a90210921c24f38eeae7eb2ce52839afa5a7     0        29982488154 lovelace + TxOutDatumNone


In [19]:
TX_0="1f573374146f20b223473329c267a90210921c24f38eeae7eb2ce52839afa5a7#0"

## 3. Design the Marlowe contract

For this demonstraction we just use a simple contract that involves two notifications and the refund of the Ada at the script address. Use the [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/#/blockly) to create this contract and the "Download JSON" button to access the JSON representation of the contract.

![Blockly for the two-notify contract](simple-babbage.png)

We customize the downloaded JSON with our own timeout values.

In [20]:
TIMEOUT=$((NOW + 30 * DAY))
echo $TIMEOUT

1663600744000


### 3.1. Creation

The initial contract has two notifications.

In [21]:
cat > simple-babbage-1.contract << EOI
{
  "when" : [
    {
      "case" : { "notify_if" : true },
      "then" : {
        "when" : [
          {
            "case" : { "notify_if" : true },
            "then" : "close"
          }
        ],
        "timeout": $TIMEOUT,
        "timeout_continuation" : "close"
      }
    }
  ],
  "timeout" : $TIMEOUT,
  "timeout_continuation" : "close"
}
EOI
json2yaml simple-babbage-1.contract

timeout: 1663600744000
timeout_continuation: close
when:
- case:
    notify_if: true
  then:
    timeout: 1663600744000
    timeout_continuation: close
    when:
    - case:
        notify_if: true
      then: close


We set the initial state to hold a small amount of Ada.

In [22]:
ACCOUNT_LOVELACE=$((3 * ADA))

In [23]:
cat << EOI > simple-babbage-1.state
{
  "accounts" : [
    [
      [
        { "pk_hash" : "$PUBKEYHASH_P" },
        { "currency_symbol" : "", "token_name" : "" }
      ],
      $ACCOUNT_LOVELACE
    ]
  ],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml simple-babbage-1.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1661008744000


### 3.2. First notification

We send an `INotify` to the contract.

In [24]:
marlowe-cli input notify > simple-babbage-1.input
json2yaml simple-babbage-1.input

input_notify
...


The contract then becomes much simpler.

In [25]:
cat > simple-babbage-2.contract << EOI
{
  "when" : [
    {
      "case" : { "notify_if" : true },
      "then" : "close"
    }
  ],
  "timeout" : $TIMEOUT,
  "timeout_continuation" : "close"
}
EOI
json2yaml simple-babbage-2.contract

timeout: 1663600744000
timeout_continuation: close
when:
- case:
    notify_if: true
  then: close


We plan to submit the input within the following validity range.

In [26]:
INVALID_BEFORE=$(time2slot $NOW)
INVALID_HEREAFTER=$(time2slot $((NOW + 5 * HOUR)))
echo "$INVALID_BEFORE $INVALID_HEREAFTER"

1005544 1023544


This means that the state after the first input will be the same as the previous state, since no adjustment of `minTime` occurs due to this validity range.

In [27]:
cat << EOI > simple-babbage-2.state
{
  "accounts" : [
    [
      [
        { "pk_hash" : "$PUBKEYHASH_P" },
        { "currency_symbol" : "", "token_name" : "" }
      ],
      $ACCOUNT_LOVELACE
    ]
  ],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml simple-babbage-2.state

accounts:
- - - pk_hash: 0a11b0c7e25dc5d9c63171bdf39d9741b901dc903e12b4e162348e07
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1661008744000


### 3.3. Second notification and closure

We send another `INotify` to the contract, which causes it to close.

In [28]:
marlowe-cli input notify > simple-babbage-2.input
json2yaml simple-babbage-2.input

input_notify
...


The contract just becomes `Close`.

In [29]:
cat > simple-babbage-3.contract << EOI
"close"
EOI
json2yaml simple-babbage-3.contract

close
...


The state becomes empty.

In [30]:
cat << EOI > simple-babbage-3.state
{
  "accounts" : [],
  "choices" : [],
  "boundValues" : [],
  "minTime" : $NOW
}
EOI
json2yaml simple-babbage-3.state

accounts: []
boundValues: []
choices: []
minTime: 1661008744000


## 4. Run the transactions

### 4.1. Creation

Bundle the contract and state, computing the datum and script.

In [31]:
marlowe-cli contract marlowe --testnet-magic "$MAGIC"                  \
                             --contract-file simple-babbage-1.contract \
                             --state-file    simple-babbage-1.state    \
                             --out-file      simple-babbage-1.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}
Validator size: 12582
Datum size: 132
Redeemer size: 1
Total size: 12715


Extract the Plutus script and its address.

In [32]:
jq '.validator.script' simple-babbage-1.marlowe > simple-babbage.plutus
head -c 1000 simple-babbage.plutus

{
  "type": "PlutusScriptV1",
  "cborHex": "5931265931230100003323232332232323232323232323232323232323233223232323232323232323232323233223232323232323232323233223322323232323232332232323232323232323232323233223322323232323232323232323232323232323232323232323232323232323232323232323232332232323232323232323232323232323232323232232222323253353332223500a2235005232322350072323232323223353235001223500223223355335333573466e2000400c23404230044c0d0c8488c00400ccd542180400c00454cd4ccd5cd19b88001501008d0108c0113034332212233002004003501033550860100300113322122330010040033355086015002001350112222333308901004003002500623033122222300200622533335333333305308e0100200101000650a00150a001130341222220051303412222200313034122222004222221533500513333038004003002001153333335015221303b03c13501822225335333355307612001505e2209901004098011303d03e1333303c0080070060052221303c03d2221303c03d222221303e03f2221303c03d15335333573466e24005403822c0422804540384004cc8848cc00400c008d4d401c888888888801088d4008894ccd400884d40108

In [33]:
ADDRESS_S=$(jq -r '.validator.address' simple-babbage-1.marlowe)
echo $ADDRESS_S

addr_test1wp3v2mx0ccsh4l6kjtsa86lgnsss20f3lsgcst9jr07axpcmajgcl


Extract the datum.

In [34]:
jq '.datum.json' simple-babbage-1.marlowe > simple-babbage-1.datum

Build the transaction

In [35]:
cardano-cli transaction build --testnet-magic "$MAGIC"                           \
                              --babbage-era                                      \
                              --protocol-params-file simple-babbage.protocol     \
                              --tx-in "$TX_0"                                    \
                              --tx-out "$ADDRESS_S+$ACCOUNT_LOVELACE"            \
                                --tx-out-datum-embed-file simple-babbage-1.datum \
                              --change-address "$ADDRESS_P"                      \
                              --out-file tx-1.raw

Estimated transaction fee: Lovelace 174785


In [36]:
cardano-cli transaction sign --testnet-magic "$MAGIC"           \
                             --tx-body-file tx-1.raw            \
                             --signing-key-file "$PAYMENT_SKEY" \
                             --out-file tx-1.signed

In [37]:
cardano-cli transaction submit --testnet-magic "$MAGIC" \
                               --tx-file tx-1.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

In [38]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_S

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7b545c4bd02910944c6bfef6a4f7110288a514dd56bd29716294139bab24e860     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "cdd48c170249c837569d17afe4d32f13c54d0f0ae3ad9f43c140039ad1747e27"


Record the transaction ID.

In [39]:
TX_1=7b545c4bd02910944c6bfef6a4f7110288a514dd56bd29716294139bab24e860

See that the change has been received.

In [40]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_P

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7b545c4bd02910944c6bfef6a4f7110288a514dd56bd29716294139bab24e860     0        29979313369 lovelace + TxOutDatumNone


### 4.2. First notification

Bundle the contract and state, computing the redeemer, datum, and script.

In [41]:
marlowe-cli contract marlowe --testnet-magic "$MAGIC"                  \
                             --input-file    simple-babbage-1.input    \
                             --contract-file simple-babbage-2.contract \
                             --state-file    simple-babbage-2.state    \
                             --out-file      simple-babbage-2.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}
Validator size: 12582
Datum size: 102
Redeemer size: 9
Total size: 12693


Extract the redeemer.

In [42]:
jq '.redeemer.json' simple-babbage-2.marlowe > simple-babbage-1.redeemer

Extract the datum.

In [43]:
jq '.datum.json' simple-babbage-2.marlowe > simple-babbage-2.datum

Build the transaction

In [44]:
cardano-cli transaction build --testnet-magic "$MAGIC"                           \
                              --babbage-era                                      \
                              --protocol-params-file simple-babbage.protocol     \
                              --tx-in-collateral "$TX_1#0"                       \
                              --tx-in "$TX_1#0"                                  \
                              --tx-in "$TX_1#1"                                  \
                                --tx-in-script-file   simple-babbage.plutus      \
                                --tx-in-datum-file    simple-babbage-1.datum     \
                                --tx-in-redeemer-file simple-babbage-1.redeemer  \
                              --tx-out "$ADDRESS_S+$ACCOUNT_LOVELACE"            \
                                --tx-out-datum-embed-file simple-babbage-2.datum \
                              --change-address "$ADDRESS_P"                      \
                              --required-signer "$PAYMENT_SKEY"                  \
                              --invalid-before "$INVALID_BEFORE"                 \
                              --invalid-hereafter "$INVALID_HEREAFTER"           \
                              --out-file tx-2.raw

Estimated transaction fee: Lovelace 966624


In [45]:
cardano-cli transaction sign --testnet-magic "$MAGIC"           \
                             --tx-body-file tx-2.raw            \
                             --signing-key-file "$PAYMENT_SKEY" \
                             --out-file tx-2.signed

In [46]:
cardano-cli transaction submit --testnet-magic "$MAGIC" \
                               --tx-file tx-2.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

In [47]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_S

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f22a3ce3b4783a46160da78dc48fcd5a2433db9f235216f7d4e96a8871c5337a     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "37af45656716a7581340384c44f028980c5fe92f301523d70a96a4f6599bd5b8"


Record the transaction ID.

In [48]:
TX_2=f22a3ce3b4783a46160da78dc48fcd5a2433db9f235216f7d4e96a8871c5337a

See that the change has been received.

In [49]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_P

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f22a3ce3b4783a46160da78dc48fcd5a2433db9f235216f7d4e96a8871c5337a     0        29978346745 lovelace + TxOutDatumNone


### 4.3. Second notification

Bundle the contract and state, computing the redeemer, datum, and script.

In [50]:
marlowe-cli contract marlowe --testnet-magic "$MAGIC"                  \
                             --input-file    simple-babbage-2.input    \
                             --contract-file simple-babbage-3.contract \
                             --state-file    simple-babbage-3.state    \
                             --out-file      simple-babbage-3.marlowe  \
                             --print-stats


Bare-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}
Validator size: 12582
Datum size: 23
Redeemer size: 9
Total size: 12614


Extract the redeemer.

In [51]:
jq '.redeemer.json' simple-babbage-3.marlowe > simple-babbage-2.redeemer

No datum is needed because the contract does not continue.

Build the transaction

In [52]:
cardano-cli transaction build --testnet-magic "$MAGIC"                           \
                              --babbage-era                                      \
                              --protocol-params-file simple-babbage.protocol     \
                              --tx-in-collateral "$TX_2#0"                       \
                              --tx-in "$TX_2#0"                                  \
                              --tx-in "$TX_2#1"                                  \
                                --tx-in-script-file   simple-babbage.plutus      \
                                --tx-in-datum-file    simple-babbage-2.datum     \
                                --tx-in-redeemer-file simple-babbage-2.redeemer  \
                              --tx-out "$ADDRESS_P+$ACCOUNT_LOVELACE"            \
                              --change-address "$ADDRESS_P"                      \
                              --required-signer "$PAYMENT_SKEY"                  \
                              --invalid-before "$INVALID_BEFORE"                 \
                              --invalid-hereafter "$INVALID_HEREAFTER"           \
                              --out-file tx-3.raw

Estimated transaction fee: Lovelace 900923


Sign and submit the transaction.

In [53]:
cardano-cli transaction sign --testnet-magic "$MAGIC"           \
                             --tx-body-file tx-3.raw            \
                             --signing-key-file "$PAYMENT_SKEY" \
                             --out-file tx-3.signed

In [54]:
cardano-cli transaction submit --testnet-magic "$MAGIC" \
                               --tx-file tx-3.signed

Transaction successfully submitted.


Wait for the transaction to be confirmed. (One can check the node logs to see when the confirmation has occurred.)

See that there is no UTxO at the script address.

In [55]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_S

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


See that the payment and change have been received.

In [56]:
cardano-cli query utxo --testnet-magic $MAGIC --address $ADDRESS_P

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c8fe8d22c7ec17272a6387982a349e8614b0f4a5d526d64f54c1bc96a4cdd52e     0        29977445822 lovelace + TxOutDatumNone
c8fe8d22c7ec17272a6387982a349e8614b0f4a5d526d64f54c1bc96a4cdd52e     1        3000000 lovelace + TxOutDatumNone
